In [1]:
# !pip install ragas langchain langchain-openai

In [2]:
import pandas as pd

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_text_splitters import TokenTextSplitter
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from ragas import evaluate
from ragas.testset import TestsetGenerator
from ragas.metrics import LLMContextRecall, LLMContextPrecisionWithReference, Faithfulness, AnswerRelevancy
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from src.utils.config import ConfigDB, ConfigLLM
from src.prompts import ANALYSIS_SYSTEM_PROMPT
from src.retrievals.search_agent import execute_dual_query_search


/Users/kim/SKN/SKN21-4th-4Team/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_10448/362612864.py:13: DeprecationWarning: Importing LLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextRecall
  from ragas.metrics import LLMContextRecall, LLMContextPrecisionWithReference, Faithfulness, AnswerRelevancy
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_10448/362612864.py:13: DeprecationWarning: Importing LLMContextPrecisionWithReference from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContext

In [3]:
import os
from dotenv import load_dotenv

# .env 파일 로드 (파일이 없으면 무시됨)
load_dotenv()
# .env.local 파일 로드 (로컬 설정 오버라이드)
load_dotenv('.env.local', override=True)

True

In [4]:
from src.utils.config import ConfigDB, ConfigLLM

COLLECTION_NAME = ConfigDB.COLLECTION_NAME
EMBEDDING_MODEL = ConfigDB.EMBEDDING_MODEL
VECTOR_SIZE = ConfigDB.VECTOR_SIZE
OPENAI_MODEL = ConfigLLM.OPENAI_MODEL

COLLECTION_NAME, EMBEDDING_MODEL, VECTOR_SIZE, OPENAI_MODEL

('learning_ai', 'text-embedding-3-large', 3072, 'gpt-4o-mini')

In [6]:
# 데이터셋을 생성할 때 사용할 context를 추출 - sampling
client = QdrantClient(host='localhost', port=ConfigDB.PORT)

# 전체 데이터를 다 조회해서 그 중 랜덤하게 K개만 sampling
info = client.get_collection(ConfigDB.COLLECTION_NAME)
total_count = info.points_count

print("info : ", info)
print("total_count : ", total_count)

info :  status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> warnings=None indexed_vectors_count=10976 points_count=11526 segments_count=5 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=3072, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None, inline_storage=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahe

In [7]:
results, next_id = client.scroll(
    collection_name=COLLECTION_NAME,
    limit=info.points_count,
)

# sampling
import random
sample_dataset = random.sample(results, 5) # 리스트에서 랜덤하게 K개를 추출


# 문서 내용만 추출
docs = [point.payload['page_content'] for point in sample_dataset]
docs

['[TITLE] secrets\n[H1] secrets --- Generate secure random numbers for managing secrets\n\nThe secrets module is used for generating cryptographically strong\nrandom numbers suitable for managing data such as passwords, account\nauthentication, security tokens, and related secrets.\n\nIn particular, secrets should be used in preference to the\ndefault pseudo-random number generator in the random module, which\nis designed for modelling and simulation, not security or cryptography.',
 '[TITLE] xml.sax.handler\n[H1] xml.sax.handler --- Base classes for SAX handlers\n\nclass DTDHandler\n\n   Handle DTD events.\n\n   This interface specifies only those DTD events required for basic parsing\n   (unparsed entities and attributes).\n\nclass EntityResolver\n\n   Basic interface for resolving entities. If you create an object implementing\n   this interface, then register the object with your Parser, the parser will call\n   the method in your object to resolve all external entities.\n\nclass E

In [8]:
#################################################################
# Vector DB 연결
# retriever 생성
#################################################################

def get_vectorstore(collection_name: str):
    # 1. 임베딩 모델 설정
    dense_embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
    # 2. Qdrant 클라이언트 연결
    client = QdrantClient(host="localhost", port=6333)
    # 3. 기존 컬렉션에 연결
    vectorstore = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=dense_embeddings
    )
    
    return vectorstore


def get_retriever(vectorstore, k: int = 5):
    # Vanilla Retriever 대신 search_agent의 고급 검색 로직 사용
    def advanced_retriever_fn(query: str):
        # 1. 듀얼 쿼리 + 하이브리드 검색 + 리랭킹 실행
        results, info = execute_dual_query_search(query)
        
        # 2. 결과 형식을 LangChain Document로 변환
        documents = []
        for r in results:
            documents.append(Document(
                page_content=r['content'],
                metadata=r.get('metadata', {})
            ))
        return documents
    return advanced_retriever_fn


In [9]:
vectorstore = get_vectorstore(COLLECTION_NAME)
retriever = get_retriever(vectorstore)

In [12]:
################################################################################
# 평가할 RAG Chain
################################################################################
prompt_txt = ANALYSIS_SYSTEM_PROMPT
prompt = ChatPromptTemplate.from_template(
    template=prompt_txt
)


def format_docs(documents:list)->str:
    """
    VectorStore에 조회한 문서들에서 내용(page_content)만 추출해서 str으로 합쳐서 반환.
    VectorStore의 검색결과인 List[Document]를 받아서 Document들에서 page_content의 내용만 추출한다.
    
    Args:
        documents(list[Document]): [Document(..), Document(...), ..]}
    Returns:
        str: 각 문서의 내용을 "\n\n"으로 연결한 string
    """
    return "\n\n".join(doc.page_content for doc in documents)

model = ChatOpenAI(model=OPENAI_MODEL)
parser = StrOutputParser()

# RAG 평가를 위해서 "답변", "검색한 문서" 둘이 출력되도록 변경.
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter

def format_doc_list(docs_dict: dict) -> list:
    # dictionary[list[Document], query:str] -> list[str]  
    # 문서내용만 추출해서(Document.page_content)만 추출한 리스트
    return [doc.page_content for doc in docs_dict['context']]

# dict | dict -> dict
# RunnablePassthrough() | dict | dict ==> RunnableSequence
chain = RunnablePassthrough() | {
    "context":retriever,
    "query":RunnablePassthrough()
} | {
    "response": prompt | model | parser,
    "retrieved_context": format_doc_list, # RAGAS 평가시 context -> List[str]
}

In [13]:
res = chain.invoke("파인튜닝의 장점은 무엇인가?")
res

ResponseHandlingException: [Errno 8] nodename nor servname provided, or not known

In [14]:
llm_context = """당신은 Python 프로그래밍 교육 과정의 **AI 조교(Teaching Assistant)**이자 데이터셋 생성 전문가입니다.
제공된 [강의 자료(Lecture)]와 [Python 공식 문서(RST)]를 기반으로, 학습자 평가를 위한 고품질의 JSON 포맷 질의응답(QA) 데이터셋을 생성하십시오.

생성 시 다음의 **지침(Instruction)과 답변 스타일(Tone & Style)**을 철저히 준수해야 합니다.

---

## Instruction
당신은 실제 강의를 보조하는 **AI 조교(Teaching Assistant) Agent**이다.  
search agent가 제공한 **강의 자료(context)** 를 바탕으로,  
수강생의 질문에 대해 **강의 흐름 안에서 이해를 돕는 답변**을 제공한다.

---

## 기본 원칙

1. **질문 중심**
   - 사용자의 질문 의도를 먼저 파악하고, 그 질문에 직접적으로 필요한 내용만 답변한다.
   - 질문에 포함되지 않은 개념 확장이나 일반적인 배경 설명은 하지 않는다.

2. **강의 맥락 우선**
   - 모든 설명은 제공된 강의 자료(context)의 내용과 흐름을 기준으로 한다.
   - 내부 강의 자료([Original])를 최우선으로 사용하며, 외부 자료([External Web])가 있는 경우 보조적으로만 참고한다.

3. **불필요한 정보 배제**
   - 질문과 직접적인 관련이 없는 내용은 포함하지 않는다.
   - 설명이 가능하더라도 질문 범위를 벗어난다면 생략한다.

4. **정직한 한계 표현**
   - 질문에 대한 직접적인 근거가 강의 자료에 없는 경우,
     “강의 자료에서는 해당 내용이 직접적으로 다뤄지지 않는다”고 명확히 말한다.
   - 다만, 강의 흐름상 최소한의 맥락 설명이 필요한 경우에만 짧게 보충한다.

---

## 답변 방식

- 먼저 **질문에 대한 핵심 답**을 간단히 제시한다.
- 이후 필요할 경우에만:
  - 강의 자료의 어느 부분과 연결되는지
  - 코드나 실습이 있다면 그 목적과 역할
  을 설명한다.
- 질문이 단순한 경우에는 추가 설명 없이 핵심 답변만 제공해도 된다.

---

## Output Guide (유연 적용)

필요한 경우에만 아래 요소를 포함한다:

- **핵심 답변**: 질문에 대한 직접 관련된 핵심 개념을 간단히 제시한다.
- **강의 맥락 설명**: 강의 자료에서 어떤 파일 / 어떤 코드 또는 셀과 연결되는지 명확히 설명한다.
- **실습/코드 연결**: 왜 이 코드가 등장했는지에 대한 설명을 수업 흐름 기준으로 설명한다.
- **한 줄 정리**: 시험 대비 또는 복습용으로 한 문장으로 요약한다.

모든 요소를 항상 포함할 필요는 없다.

---

## Tone & Style

- 실제 강의 조교처럼 **차분하고 설명 중심적인 말투**를 사용한다.
- 과장, 불필요한 비유, 감정 표현은 사용하지 않는다.
- 질문에 대한 답변이 자연스럽게 끝나면 추가 문장은 붙이지 않는다.

---

## QA 생성 제약 (Constraints)
- **언어**: 가능하면 한국어로 질문하고 답변하되, 코드 예제는 영어로 작성하십시오.
- **문법**: 문장의 끝은 마침표(.) 등 **구두점을 반드시 표기**하여 완결된 문장으로 작성하십시오.
- **JSON 포맷 엄수**:
  - 출력은 반드시 파싱 가능한 **JSON 배열** 형태여야 합니다 (`[{"user_input": "...", "reference": "..."}, ...]`).
  - **이스케이프 처리**: 본문 내의 큰따옴표("), 백슬래시(\) 등 특수 문자는 반드시 역슬래시(\")를 사용하여 이스케이프 처리하십시오.
  - 소스 텍스트에 JSON 문법을 해치는 요소가 있더라도, 최종 출력은 유효한 JSON이어야 합니다.

---

## QA 생성 출력 예시
[
  {
    "user_input": "파인튜닝의 장점은 무엇인가?",
    "reference": "파인튜닝은 사전 학습된 모델(Foundation 모델)을 특정 태스크나 도메인 데이터로 추가 학습 최적화 하는 과정으로, 특정 작업/도메인에 최적화, 사용자 맞춤형 톤앤매너 적용, 안전성과 윤리 강화를 위해 필요하다."
  }
]

위 가이드를 바탕으로 주어진 텍스트에서 학습 가치가 높은 QA 세트를 생성하십시오.
"""

<>:73: SyntaxWarning: invalid escape sequence '\)'
<>:73: SyntaxWarning: invalid escape sequence '\)'
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_10448/2424486145.py:73: SyntaxWarning: invalid escape sequence '\)'
  - **이스케이프 처리**: 본문 내의 큰따옴표("), 백슬래시(\) 등 특수 문자는 반드시 역슬래시(\")를 사용하여 이스케이프 처리하십시오.


In [15]:
generator_llm = LangchainLLMWrapper(ChatOpenAI(model=OPENAI_MODEL))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=EMBEDDING_MODEL))

generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    llm_context=llm_context 
)

# testset
testset = generator.generate_with_chunks(
    docs, testset_size=20  # context 내용, 테스트데이터셋 몇개를 만들지.
)
testset

/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_10448/496705960.py:1: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  generator_llm = LangchainLLMWrapper(ChatOpenAI(model=OPENAI_MODEL))
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_10448/496705960.py:2: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=EMBEDDING_MODEL))
Generating Samples: 100%|██████████| 20/20 [00:15<00:00,  1.30it/s]


Testset(samples=[TestsetSample(eval_sample=SingleTurnSample(user_input='How does the secrets module enhance the security of passwords?', retrieved_contexts=None, reference_contexts=['[TITLE] secrets\n[H1] secrets --- Generate secure random numbers for managing secrets\n\nThe secrets module is used for generating cryptographically strong\nrandom numbers suitable for managing data such as passwords, account\nauthentication, security tokens, and related secrets.\n\nIn particular, secrets should be used in preference to the\ndefault pseudo-random number generator in the random module, which\nis designed for modelling and simulation, not security or cryptography.'], retrieved_context_ids=None, reference_context_ids=None, response=None, multi_responses=None, reference='The secrets module is used for generating cryptographically strong random numbers suitable for managing data such as passwords, ensuring enhanced security compared to the default pseudo-random number generator in the random mo

In [ ]:
eval_df = testset.to_pandas()
eval_df.head()

,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name
0,How does the secrets module enhance the securi...,[[TITLE] secrets\n[H1] secrets --- Generate se...,The secrets module is used for generating cryp...,Data Security Engineer,WEB_SEARCH_LIKE,MEDIUM,single_hop_specific_query_synthesizer
1,What is the purpose of the secrets module in P...,[[TITLE] secrets\n[H1] secrets --- Generate se...,The secrets module is used for generating cryp...,Data Security Engineer,PERFECT_GRAMMAR,SHORT,single_hop_specific_query_synthesizer
2,How does the secrets module enhance account au...,[[TITLE] secrets\n[H1] secrets --- Generate se...,The secrets module is used for generating cryp...,Data Security Engineer,WEB_SEARCH_LIKE,LONG,single_hop_specific_query_synthesizer
3,What random numbers for secrets?,[[TITLE] secrets\n[H1] secrets --- Generate se...,The secrets module is used for generating cryp...,Data Security Engineer,POOR_GRAMMAR,SHORT,single_hop_specific_query_synthesizer
4,What is the role of the EntityResolver in XML ...,[[TITLE] xml.sax.handler\n[H1] xml.sax.handler...,The EntityResolver is a basic interface for re...,Python Developer,WEB_SEARCH_LIKE,MEDIUM,single_hop_specific_query_synthesizer


In [17]:
# >>>> Chain 응답들을 저장할 list
response_list = []
# Chain 이 반환한 context들을 지정할 list
retrieved_context_list = []

for user_input in eval_df['user_input']:
    res = chain.invoke(user_input)
    response_list.append(res['response'])
    retrieved_context_list.append(res['retrieved_context'])

# >>>> eval_df에 응답과 context 추가
eval_df['response'] = response_list
eval_df['retrieved_contexts'] = retrieved_context_list
eval_df.head()

ResponseHandlingException: [Errno 8] nodename nor servname provided, or not known

In [ ]:
from ragas import EvaluationDataset
# from_xxxx() xxxx 타입의 객체를 EvaluationDataset객체로 변환.
eval_dataset = EvaluationDataset.from_pandas(
    eval_df[["user_input", "retrieved_contexts", "response", "reference"]]
)
eval_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=20)

In [ ]:
eval_llm = LangchainLLMWrapper(ChatOpenAI(model=OPENAI_MODEL))
eval_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=EMBEDDING_MODEL))
# 평가할 함수들을 List로 묶어준다.
metrics = [
    LLMContextRecall(llm=eval_llm),
    LLMContextPrecisionWithReference(llm=eval_llm),
    Faithfulness(llm=eval_llm),
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embeddings)
]

# Run Evaluation
eval_results = evaluate(dataset=eval_dataset, metrics=metrics)

# Convert to Pandas DataFrame for easier viewing
df_results = eval_results.to_pandas()
display(df_results.head(10))

/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_8456/3596288186.py:1: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  eval_llm = LangchainLLMWrapper(ChatOpenAI(model=OPENAI_MODEL))
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_8456/3596288186.py:2: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  eval_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=EMBEDDING_MODEL))
Evaluating:  22%|██▎       | 18/80 [00:48<01:59,  1.93s/it]LLM returned 1 generations instead of requested 3. Proceeding 

,user_input,retrieved_contexts,response,reference,context_recall,llm_context_precision_with_reference,faithfulness,answer_relevancy
0,페르소나의 역할 정의는 무엇인가요?,"[[강의: 05_함수]\n\n함수란 \n- 프로그램에서 함수란 하나의 작업, 기능...",페르소나의 역할 정의는 모델에게 명확한 역할이나 페르소나를 부여하여 일관된 응답을 ...,"페르소나(persona, 역할)는 모델에게 명확한 역할을 부여하여 일관된 응답을 얻...",1.0,0.333333,1.000000,0.991149
1,초등학교 과학 선생님으로서 물의 순환 과정을 어떻게 쉽게 설명할 수 있나요?,[[강의: 03_Prompt]\n\n| 국가 | 인구 (백만명) |\n|------...,"핵심 답변: 초등학교 과학 선생님으로서 물의 순환 과정을 쉽게 설명하기 위해, 물이...","물의 순환 과정은 물이 지구에서 어떻게 이동하는지를 설명합니다. 먼저, 태양의 열로...",1.0,1.000000,0.375000,0.697705
2,비즈니스 분석 보고서를 작성하는 단계는 무엇인가요?,[[강의: 03_Prompt]\n\n| 국가 | 인구 (백만명) |\n|------...,핵심 답변: 비즈니스 분석 보고서를 작성하는 단계는 다음과 같습니다:\n1. 데이터...,"비즈니스 분석 보고서를 작성하는 단계는 다음과 같습니다: 1단계: 데이터 검토, 2...",1.0,0.833333,1.000000,0.695498
3,"기술 낙관주의자란 무엇이며, 그들의 주장은 어떤 특징을 가지고 있나요?",[[강의: 01_자연어처리(NLP)개요]\n\n자연어 처리(NLP) 개요 \n**...,강의 자료에서는 ‘기술 낙관주의자’에 대한 직접적인 설명이 다뤄지지 않습니다. 따라...,기술 낙관주의자는 기술의 발전이 사회에 긍정적인 영향을 미친다고 믿는 사람들입니다....,0.0,0.000000,0.666667,0.000000
4,DB API랑 csv 파일 읽고 쓰기랑 어떻게 연결되나? 이거 좀 자세히 설명해줘.,[[API] The so-called CSV (Comma Separated Valu...,**핵심 답변**: DB API와 CSV 파일 읽기 및 쓰기는 데이터를 서버와 클라...,"DB API와 csv 파일 읽고 쓰기는 서로 연결될 수 있는데, DB API를 통해...",1.0,0.866667,0.666667,0.758927
5,How does the DB API facilitate the integration...,[[TITLE] csv\n[H1] csv --- CSV File Reading an...,핵심 답변: DB API는 SQL NULL 데이터 값을 CSV 파일에 통합하는 것을...,The DB API facilitates the integration of SQL ...,1.0,0.755556,0.250000,0.837454
6,How does the DB API facilitate the integration...,[[TITLE] csv\n[H1] csv --- CSV File Reading an...,"DB API는 SQL NULL 데이터 값을 CSV 파일에 통합하는 데 있어서, NU...",The DB API facilitates the integration of SQL ...,1.0,0.805556,0.500000,0.978957
7,DB API랑 csv 파일 어떻게 연결되나?,[[API] The so-called CSV (Comma Separated Valu...,DB API와 CSV 파일은 데이터의 이동 및 처리를 담당하는 두 가지 다른 기능을...,DB API를 사용하는 모듈은 SQL NULL 데이터 값을 CSV 파일로 덤프할 때...,0.0,0.700000,0.454545,0.698442
8,What is the purpose of collections.abc.AsyncGe...,[[KEYWORDS] __aiter__ __anext__ __buffer__\n[T...,**핵심 답변**: `collections.abc.AsyncGenerator`는 비...,The collections.abc.AsyncGenerator is a class ...,1.0,0.500000,1.000000,0.921548
9,Awatable의 정의는 무엇인가요?,[[강의: 11_seq2seq_attention_chatbot]\n\nAttenti...,"강의 자료에서는 ""Awatable""에 대한 정의가 직접적으로 다뤄지지 않습니다. 따...",Awaitable은 collections.abc.Awaitable의 deprecat...,0.0,0.000000,0.666667,0.000000


- **Retrieval**
    -  질문에 대해 검색한 문서(context)들에 대한 평가
    -  **Context Precision(문맥 정밀도)**
        -  검색된 문서(context)들 중 질문과 관련 있는 것들이 **얼마나 상위 순위에 위치하는지** 평가하는 지표.
    -  **Context Recall(문맥 재현률)**
        -  검색된 문서(context)가 정답(ground-truth)의 정보를 얼마나 포함하고 있는지 평가하는 지표.
- **Generation**
    - llm 모델이 생성한 답변에 대한 평가 지표들.
    - **Faithfulness(신뢰성)**
        -  생성된 답변과 검색된 문서(context)간의 관련성을 평가하는 지표
        -  생성된 답변이 주어진 문맥(context)에 얼마나 충실한지를 평가하는 지표로 할루시네이션에 대한 평가로 볼 수있다.
    - **Answer relevancy(답변 적합성)**
        - 생성된 답변과 사용자의 질문간의 관련성을 평가하는 지표
        - 생성된 답변이 사용자의 질문과 얼마나 관련성이 있는지를 평가하는 지표.

In [ ]:
print("\n📊 Evaluation Results:")
print(eval_results)


📊 Evaluation Results:
{'context_recall': 0.6250, 'llm_context_precision_with_reference': 0.5083, 'faithfulness': 0.7121, 'answer_relevancy': 0.5868}


In [ ]:
eval_results.to_pandas()
eval_results

{'context_recall': 0.6250, 'llm_context_precision_with_reference': 0.5083, 'faithfulness': 0.7121, 'answer_relevancy': 0.5868}